In [27]:
import sys
import os

# Add the project root to sys.path
sys.path.append(os.path.abspath(".."))

from app.services import (
    extract_top_n_nouns_with_frequency,
    get_sentences_with_target_word
)

In [28]:
with open('../static/uploads/Owczarek.txt') as f:
    text_content = f.read()

print(len(text_content))

18552


In [29]:
extract_top_n_nouns_with_frequency(text_content, 10)

{'life': 58,
 'qol': 40,
 'quality': 38,
 'health': 29,
 'concept': 22,
 'patients': 20,
 'patient': 16,
 'owczarek': 14,
 'situation': 12,
 'treatment': 11}

In [30]:
sentences = get_sentences_with_target_word(text_content, 'qol', 10)

In [31]:
len(sentences)

10

In [32]:
sentences

['SUMMARY The increasing need to assess the effects of various dis- eases on patients’ psychological and social functioning has led to a number of initiatives aimed at quantitative evaluation of quality of life (QOL).',
 'Many definitions of QOL can be found in the literature.',
 'Numerous QOL studies have endeav- oured to learn more about the wellbeing of different patient groups, and especially about the effects of various medical interventions which cannot be measured by quantitative bio- logical parameters alone.',
 'QOL is usually assessed with self-report measures.',
 'QOL assessment is becoming a standard component of comprehensive clinical assessment of patients with chronic diseases that can have a negative effect on every aspect of life.',
 'Correct medical intervention involves not only accurate diagnosis of the disease and implementation of adequate and effective pharmacotherapy, but also correct evaluation of all the components of QOL and level of satisfaction of patients’